In [16]:
import pandas as pd
import numpy as np
from keras.models import Sequential
from keras.layers import LSTM, Dense
import time
import concurrent.futures
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

In [17]:
file_path = 'Data\Binance_BTCUSDT_2024_minute.csv'
df = pd.read_csv(file_path)
#print(df.head())

In [18]:
df = df.drop('unix', axis=1).drop('date', axis=1).drop('symbol', axis=1).drop('volume_from', axis=1) 
#df

In [19]:
df = df.iloc[::-1]
df

,open,high,low,close,volume,tradecount
69071,42298.62,42320.00,42298.61,42320.00,21.16779,1348
69070,42319.99,42331.54,42319.99,42325.50,21.60391,1019
69069,42325.50,42368.00,42325.49,42367.99,30.50730,1241
69068,42368.00,42397.23,42367.99,42397.23,46.05107,1415
69067,42397.22,42409.20,42385.26,42409.20,32.26766,1255
...,...,...,...,...,...,...
4,51849.99,51888.89,51849.99,51870.64,36.86096,653
3,51870.63,51870.64,51841.10,51841.10,11.01314,495
2,51841.11,51847.32,51841.10,51847.31,11.83013,423
1,51847.31,51851.19,51829.20,51829.20,18.49469,506


In [20]:
df = df.dropna()

# Iterate over each column except for non-numeric columns
for column in df.columns:
    if pd.api.types.is_numeric_dtype(df[column]):
        df[column] = df[column].pct_change() * 100

df = df.dropna()
print(df)

           open      high       low     close     volume  tradecount
69070  0.050522  0.027268  0.050545  0.012996   2.060300  -24.406528
69069  0.013020  0.086130  0.012996  0.100389  41.211938   21.786065
69068  0.100412  0.068991  0.100412  0.069014  50.950986   14.020951
69067  0.068967  0.028233  0.040762  0.028233 -29.930705  -11.307420
69066  0.028233  0.051616  0.048673  0.034780 -12.804430   12.350598
...         ...       ...       ...       ...        ...         ...
4      0.001176  0.062963  0.032701  0.039807  98.890644   -3.402367
3      0.039807 -0.035171 -0.017146 -0.056949 -70.122482  -24.196018
2     -0.056911 -0.044958  0.000000  0.011979   7.418320  -14.545455
1      0.011960  0.007464 -0.022955 -0.034929  56.335476   19.621749
0     -0.034910  0.033442  0.000000  0.038955  87.199569   69.169960

[69071 rows x 6 columns]


In [21]:
percent_change.describe()

,open,high,low,close,volume,tradecount
count,69071.000000,69071.000000,69071.000000,69071.000000,69071.000000,69071.000000
mean,0.000327,0.000321,0.000325,0.000327,35.399404,17.076442
std,0.079818,0.071601,0.078025,0.080214,158.116330,90.031912
min,-10.430776,-10.416333,-10.499666,-10.571400,-97.431648,-93.400576
25%,-0.027718,-0.022833,-0.020847,-0.027716,-37.583632,-28.335285
50%,0.000000,0.000000,0.000000,0.000000,-3.045659,-2.242888
75%,0.028942,0.021512,0.024010,0.028950,55.010839,36.037499
max,1.449683,1.522594,1.260864,1.512612,9695.707025,5274.907063


In [22]:
df.describe()

,open,high,low,close,volume,tradecount
count,69071.000000,69071.000000,69071.000000,69071.000000,69071.000000,69071.000000
mean,0.000327,0.000321,0.000325,0.000327,35.399404,17.076442
std,0.079818,0.071601,0.078025,0.080214,158.116330,90.031912
min,-10.430776,-10.416333,-10.499666,-10.571400,-97.431648,-93.400576
25%,-0.027718,-0.022833,-0.020847,-0.027716,-37.583632,-28.335285
50%,0.000000,0.000000,0.000000,0.000000,-3.045659,-2.242888
75%,0.028942,0.021512,0.024010,0.028950,55.010839,36.037499
max,1.449683,1.522594,1.260864,1.512612,9695.707025,5274.907063


In [28]:
df.sort_values('volume',ascending=False)

,open,high,low,close,volume,tradecount
32757,0.000000,0.103189,0.000000,0.060075,9695.707025,340.291262
58618,-0.016350,1.522594,-0.029453,1.241536,7862.029402,5274.907063
44811,0.000024,0.000000,-0.105798,-0.044706,5705.176739,513.181818
45669,-0.000024,0.000000,-0.187046,-0.158637,4680.319554,1122.009569
36123,-0.012754,0.058281,-0.006294,0.071020,3842.614066,829.652997
...,...,...,...,...,...,...
17003,-0.005767,-0.056290,0.002852,-0.000042,-96.261312,-66.479663
14475,-0.010213,-0.038995,0.000000,0.000000,-96.273400,-88.067122
38917,-0.010287,-0.010287,0.014445,0.000000,-96.378510,-51.091703
35994,0.017245,-0.002945,0.028534,-0.010070,-97.389867,-78.011118


In [37]:
class Kek():
    prope1 = "12"

    def show_prop(self):
        return self.prope1+"kek"
    

class UserModel():
    data = "12"
    data2 = "23"
    data3 = "42"
    data4 = "51"
    def getDBData(self):
        return self.data+"_saved"
    

class UserRepository():

    def saveToDB(self, user: UserModel):
        print("save process..." + user.getDBData())

In [38]:
user = UserModel()
repoo = UserRepository()


repoo.saveToDB(user)

save process...12_saved


In [42]:
def saveToDB(dbName):

    def inner(data):
        print("saveToDB", dbName, data)
        
    return inner


kek = saveToDB("mssql")
kek("user1")

saveToDB mssql user1


In [40]:
userClosure("12", 12, saveToDB)

NameError: name 'name' is not defined